In [4]:
import sys
sys.path.append('../')
from utils.process_data_online import *

In [5]:
from utils.features import *

In [22]:
def chunks_from_files(data, metadata):
    sess_data = pd.read_csv(data,header=None)
    sess_metadata = pd.read_csv(metadata, names=['start_time','category'])
    category_instances = {}
    for i, row in sess_metadata.iterrows():
        start_time = row['start_time']
        category = row['category'].strip()
        end_time = sess_metadata.iloc[i + 1]['start_time'] if i < (len(sess_metadata) - 1) else sess_data.iloc[-1][0]
        channels = sess_data.loc[(sess_data[0] >= start_time) & (sess_data[0] < end_time)]
        instance = np.array(channels)
        if category not in category_instances:
            category_instances[category] = []
        category_instances[category].append(instance)
    return category_instances

In [29]:
data_dir = '../data/emg_recordings/'
num = 14
sess = data_dir + f'test_sess{num}_data.txt'#.format{num}
sess_md = data_dir + f'test_sess{num}_metadata.txt'#.format{num}
chunk_dict = chunks_from_files(sess, sess_md)

In [30]:
chunk_dict.keys()

dict_keys(['REST', 'CLENCH FIST', 'SNAP'])

In [31]:
def compute_peaks(dict_key):
    peaks_list = []
    for chunk in chunk_dict[dict_key]:
        peaks = find_peaks(process_chunk(chunk))
        #print(peaks)
        peaks_list.append(peaks)
    overall_peaks = np.mean(np.array(peaks_list),axis=0)
    return overall_peaks

In [32]:
def compute_mav(dict_key):
    mav_list = []
    for chunk in chunk_dict[dict_key]:
        pro_chunk = process_chunk(chunk)
        mav_metric = np.mean([mean_absolute_value(pro_chunk[:,i]) for i in range(pro_chunk.shape[1])])
        mav_list.append(mav_metric)
    overall_mav = np.mean(mav_list)
    return overall_mav

In [33]:
print(compute_peaks('REST'))
print(compute_peaks('SNAP'))
print(compute_peaks('CLENCH FIST'))

[675.55594014 344.57202215 433.48606925 311.22265471]
[ 687.53747448  667.69353346  484.24792408 1638.12867602]
[521.45168877 511.75190635 464.62465744 796.07262471]


In [34]:
print(compute_mav('REST'))
print(compute_mav('SNAP'))
print(compute_mav('CLENCH FIST'))

42.41815747238079
93.43383817499735
79.33941634895322


In [21]:
data = [[1,2],[1,4],[1,6]]
print(np.mean(np.array(data),axis=0))

[1. 4.]
